# Using Neural Networks To Solve The Problem

The below code has been adapted from [this article](https://machinelearningmastery.com/text-generation-lstm-recurrent-neural-networks-python-keras/) on text generation with LSTM and [this article](https://towardsdatascience.com/multi-class-text-classification-with-lstm-1590bee1bd17) on classification with LSTM.

In [ ]:
# Gotta bring some mates